In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import dill
import pickle
import random

In [2]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

45 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
96 unique stemmed words [',', 'a', 'abrir', 'ade', 'adequado', 'advers', 'adversa', 'adverso', 'ai', 'ajud', 'ajudou', 'alguém', 'apresentam', 'bast', 'blood', 'bom', 'by', 'causam', 'com', 'como', 'comportamento', 'convers', 'dat', 'de', 'depo', 'detail', 'dia', 'dê-me', 'entry', 'faz', 'find', 'for', 'hist', 'hospit', 'i', 'id', 'incrivel', 'isso', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'mai', 'man', 'me', 'medicamento', 'mod', 'módulo', 'nearby', 'não', 'o', 'obrigado', 'of', 'oferec', 'oi', 'olá', 'op', 'os', 'pacy', 'par', 'paty', 'pharm', 'pod', 'por', 'press', 'prov', 'qua', 'qual', 'que', 'reação', 'reaçõ', 'rel', 'result', 'search', 'ser', 'show', 'suport', 'tard', 'task', 'tchau', 'te', 'tem', 'tipo', 'to', 'todo', 'transf', 'um', 'up', 'vai', 'vejo', 'verific', 'você', 'want', 'útil']


In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

/home/kaique/miniconda3/envs/av/lib/python3.6/site-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [6]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [7]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
9/9 [==============================] - 0s 1ms/step - loss: 2.1905 - accuracy: 0.1878
Epoch 2/200
9/9 [==============================] - 0s 1ms/step - loss: 2.1435 - accuracy: 0.1462
Epoch 3/200
9/9 [==============================] - 0s 1ms/step - loss: 2.1880 - accuracy: 0.1437
Epoch 4/200
9/9 [==============================] - 0s 1ms/step - loss: 2.0210 - accuracy: 0.2215
Epoch 5/200
9/9 [==============================] - 0s 1ms/step - loss: 1.9130 - accuracy: 0.4048
Epoch 6/200
9/9 [==============================] - 0s 1ms/step - loss: 1.8870 - accuracy: 0.3746
Epoch 7/200
9/9 [==============================] - 0s 1ms/step - loss: 1.7759 - accuracy: 0.5669
Epoch 8/200
9/9 [==============================] - 0s 1ms/step - loss: 1.6957 - accuracy: 0.4152
Epoch 9/200
9/9 [==============================] - 0s 1ms/step - loss: 1.4086 - accuracy: 0.7159
Epoch 10/200
9/9 [==============================] - 0s 1ms/step - loss: 1.3274 - accuracy: 0.5354
Epoch 11/200
9/9 [===========

9/9 [==============================] - 0s 1ms/step - loss: 0.0365 - accuracy: 0.9779
Epoch 84/200
9/9 [==============================] - 0s 932us/step - loss: 0.0144 - accuracy: 1.0000
Epoch 85/200
9/9 [==============================] - 0s 898us/step - loss: 0.0561 - accuracy: 1.0000
Epoch 86/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0700 - accuracy: 1.0000
Epoch 87/200
9/9 [==============================] - 0s 730us/step - loss: 0.0378 - accuracy: 1.0000
Epoch 88/200
9/9 [==============================] - 0s 919us/step - loss: 0.0283 - accuracy: 0.9931
Epoch 89/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0396 - accuracy: 1.0000
Epoch 90/200
9/9 [==============================] - 0s 824us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 91/200
9/9 [==============================] - 0s 939us/step - loss: 0.0246 - accuracy: 1.0000
Epoch 92/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0273 - accuracy: 1.0000
Epoch 93/200
9/9 [===

9/9 [==============================] - 0s 1ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 166/200
9/9 [==============================] - 0s 987us/step - loss: 0.0169 - accuracy: 0.9902
Epoch 167/200
9/9 [==============================] - 0s 865us/step - loss: 0.0092 - accuracy: 1.0000
Epoch 168/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 169/200
9/9 [==============================] - 0s 801us/step - loss: 0.0318 - accuracy: 1.0000
Epoch 170/200
9/9 [==============================] - 0s 862us/step - loss: 0.0783 - accuracy: 1.0000
Epoch 171/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 172/200
9/9 [==============================] - 0s 723us/step - loss: 0.0112 - accuracy: 1.0000
Epoch 173/200
9/9 [==============================] - 0s 971us/step - loss: 0.0563 - accuracy: 1.0000
Epoch 174/200
9/9 [==============================] - 0s 1ms/step - loss: 0.0302 - accuracy: 0.9902
Epoch 175/20

In [8]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [9]:
p = bow("Load blood pessure for patient", words)
print (p)
print (classes)

found in bag: load
found in bag: blood
found in bag: for
found in bag: paty
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [10]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])

print(model.predict(inputvar))

[[3.4171364e-07 3.7361967e-06 9.9986160e-01 2.0666393e-07 6.1206024e-06
  1.2347360e-04 1.5682486e-06 4.4551030e-07 2.5371935e-06]]


In [11]:
# save model to file
#pickle.dump(model, open("katana-assistant-model.pkl", "wb"))
#with open("katana-assistant-model.pkl", "wb") as f:
#    f.write(dill.dumps(model))
#    f.close()
model.save("katana-assistant-model.h5")

In [12]:
# save all of our data structures
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "katana-assistant-data.pkl", "wb" ) )